# Workbook explores results of the TV Test by price points

### Load packages

In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [173]:
# Load core dataset for ASINs 
df = pd.read_csv("../TV_test/asin_date_4.txt", sep="\t",
                 encoding = "ISO-8859-1", header=None, 
                 names = ['week_ending','is_promo','asin','is_promo_asin',
                          'content','title_name',
                         'studio_name','contract_id',
                          'release_date','is_hd',
                          'display_price','lifecycle','season_number','customers',
                          'new_customers','OPS','units','cogs','ppm'])

In [174]:
def initial_setup(df,content):
    """function makes basic data type transformations, 
    converts object to datetime format and renames lifecycles, gets month"""
    
    
    df['week_ending'] = pd.to_datetime(df['week_ending'])
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['month'] = df['week_ending'].dt.month
    df['new_lifecycles'] = np.where(df['lifecycle'].isin(['New Release','Recent release']),'RR','Catalogue')
    
    #Subset and work only with particular content
    df = df[df['content'] == content]
    
    return df

In [175]:
df = initial_setup(df, "Season")

In [177]:
print(df.shape)
df.head()

(158673, 21)


,week_ending,is_promo,asin,is_promo_asin,content,title_name,studio_name,contract_id,release_date,is_hd,...,lifecycle,season_number,customers,new_customers,OPS,units,cogs,ppm,month,new_lifecycles
15,2018-01-27,non_promo_period,B06W5GC1WV,non_promo_asins,Season,Feuerwehrmann Sam - Eine Insel voller Abenteuer,Hit Entertainment,4GB9Q_de,2017-03-03,Y,...,6mo_1yr_Catalogue,1.0,59,0,584.20,59,NaN,NaN,1,Catalogue
28,2018-02-17,non_promo_period,B00KX2UOGK,promo_asins,Season,Sherlock - Staffel 3 [dt./OV],Hartswood Films/BBC/MASTERPIECE,UXBB4,2014-06-10,Y,...,Deep Catalogue,3.0,158,3,1798.82,160,1255.37,0.3021,2,Catalogue
30,2018-02-17,non_promo_period,B01MXUUQCB,non_promo_asins,Season,Ein Tag in ?,Story House,UXZD4,2016-12-05,Y,...,1_2yr_Catalogue,1.0,14,0,40.98,14,25.44,0.3792,2,Catalogue
35,2018-03-03,non_promo_period,B00GNYNJ7I,promo_asins,Season,Rizzoli & Isles - Staffel 3,Warner Bros,UXWB4,2014-01-01,Y,...,Deep Catalogue,3.0,7,0,117.46,7,74.06,0.3694,3,Catalogue
39,2018-03-10,promo_period,B00ET0NHX4,promo_asins,Season,Drawn Together - Staffel 1 [dt./OV],Comedy Central,UXMT4,2014-01-01,N,...,Deep Catalogue,1.0,58,3,234.38,58,166.10,0.2913,3,Catalogue


In [178]:
#Sanity check, the revenue in the promo period of promo asins
df[(df['is_promo'] == 'promo_period')&(df['is_promo_asin'] == "promo_asins")]['OPS'].sum()

2249635.3300000001

# Distribution of promo OPS by price points

<p> Here I am doing basic exploration of the number of ASINs per price point </p>

In [179]:
def dist_by_revenue(df):
    t = pd.DataFrame(df[(df['is_promo'] == 'promo_period')
                        & (df['is_promo_asin'] == "promo_asins")].groupby("display_price")['OPS'].sum().reset_index())
    t.columns = ['display_price','OPS']
    t['share'] = t['OPS'] / t['OPS'].sum()
    t['cumsum'] = t['share'].cumsum()
    return t

In [180]:
t = dist_by_revenue(df)
print("Revenue generated from promo ASINs in the promo period:", t['OPS'].sum())
t.head()

Revenue generated from promo ASINs in the promo period: 2249635.33


,display_price,OPS,share,cumsum
0,2.49,2.09,9.290395e-07,9.290395e-07
1,2.99,12.53,5.569792e-06,6.498831e-06
2,3.49,35.16,1.562920e-05,2.212803e-05
3,3.99,2422.06,1.076646e-03,1.098774e-03
4,4.47,33.84,1.504244e-05,1.113816e-03


In [181]:
#Check the share of OPS covered by those 3 price points
t[t['display_price'].isin([4.99, 9.99, 14.99])]['OPS'].sum() / t['OPS'].sum()

0.92875611977519745

# Make adjustment to time frame for non period

<p> The step is optional: just to make the length of the periods in promo and non-promo equal </p>

In [182]:
df = df[df['week_ending'] >= '2018-01-20']

# Make a list of seasonal coefficiens

<p> This section deals with seasonality and requires another raw data file.
    The idea is to get seasonality factor for each month for RR and Catalogue </p>

In [183]:
new_data_daily = pd.read_csv("./forecast_DE_lifecycles_daily.txt", sep="\t",
                       names = ['promo_period','week_ending','transaction_date','promo_status','content','lifecycle',
                        'total_customers','new_customers','units','ops','cogs'])

In [187]:
def make_prep_for_lifecycles(df, dimension1, dimension2):
    """function makes key preparations"""
    
    df[dimension1] = pd.to_datetime(df[dimension1])
    df[dimension2] = pd.to_datetime(df[dimension2])
    df['new_lifecycles'] = np.where(df['lifecycle'].isin(['New Release','Recent release']),'RR','Catalogue')
    
    df = df[df['content'].isin(['Season'])]
    #df = df[df[dimension1] <= '2018-05-12']
    
    summed = pd.DataFrame(df.groupby([dimension1,'new_lifecycles'])['ops'].sum().reset_index())
    pivoted = pd.DataFrame(pd.pivot_table(summed, values='ops', columns='new_lifecycles', index=dimension1).reset_index())
    pivoted = pivoted.fillna(0)
    return pivoted

In [188]:
def prep_seasonality_monthly(df):
    """prepares seasonality adjustments and calls another function"""
    
    prep = make_prep_for_lifecycles(df,"transaction_date","transaction_date")
    
    prep['total'] = prep['Catalogue'] + prep['RR']
    prep['month'] = prep['transaction_date'].dt.month
    prep['year']  = prep['transaction_date'].dt.year
    #prep['mnth_yr'] = prep['transaction_date'].apply(lambda x: x.strftime('%B-%Y'))   
    
    t = pd.DataFrame(prep.groupby(['month','year'])['Catalogue','RR','total'].sum().reset_index())
    t = t[t['year'] >= 2015]
    
    t = t.sort_values(['year','month'], ascending = [True,True]).reset_index()
    t = t.drop('index', axis = 1)
    
    t['total_new'] = np.where((t['year'] == 2017) & (t['month'] ==7), t[(t['year'] == 2017) & (t['month'] < 7 )]['total'].mean(), t['total'])
    t['RR_new'] = np.where((t['year'] == 2017) & (t['month'] ==7), t[(t['year'] == 2017) & (t['month'] < 7)]['RR'].mean(), t['RR'])
    t['Catalogue_new'] = np.where((t['year'] == 2017) & (t['month'] ==7), t[(t['year'] == 2017) & (t['month'] < 7)]['Catalogue'].mean(), t['Catalogue'])

    
    return t

In [189]:
t = prep_seasonality_monthly(new_data_daily)

In [194]:
def combine_seasonality():
    def get_shape(df,dimension):  
        monthly_mean = t[t['year'] >= 2017].groupby("month")[dimension].mean()
        all_mean = t[t['year'] >= 2017][dimension].mean()
        l = pd.DataFrame({"monthly_mean":monthly_mean,"all_mean":all_mean}).reset_index()
        l['factor'] = l['monthly_mean'] / l['all_mean']
        l = l.drop(['all_mean',"monthly_mean"], axis = 1)
        l.columns = ['month', dimension]
        return l
    
    l1 = get_shape(t,"total_new")
    l2 = get_shape(t,"RR_new")
    l3 = get_shape(t,"Catalogue_new")
    
    g1 = pd.merge(left=l1,right=l2,left_on="month",right_on="month",how='left')
    g2 = pd.merge(left=g1,right=l3,left_on="month",right_on="month", how = "left")
    
    g2.columns = ['month','total','RR','Catalogue']
    return g2

In [195]:
sindex = combine_seasonality()

In [196]:
sindex

,month,total,RR,Catalogue
0,1,1.131508,1.070130,1.167832
1,2,0.969689,1.008208,0.946893
2,3,0.956748,0.900552,0.990004
3,4,0.884616,0.829465,0.917254
4,5,0.878059,0.812552,0.916825
5,6,0.800869,0.880618,0.753674
6,7,0.798610,0.778905,0.810272
7,8,1.267936,1.411177,1.183166
8,9,1.158129,1.229705,1.115771
9,10,1.165321,1.261044,1.108673


In [197]:
def get_melt(df):
    t = pd.melt(sindex, id_vars=['month'])
    t.columns = ['month','new_lifecycles','factor']
    return t

In [198]:
sindex_melt = get_melt(sindex)

In [199]:
sindex_melt.head()

,month,new_lifecycles,factor
0,1,total,1.131508
1,2,total,0.969689
2,3,total,0.956748
3,4,total,0.884616
4,5,total,0.878059


## Get ASINs in promo at these price points in the non promo period 

<p> The idea is to compare ASINs which are at this price points in the promo and non promo period </p>

In [201]:
def get_asin_at_price_points(df):
    """function selects asins that had certain price points
    and selects from pre period the same ones to make proper comparison"""
    asin_list = df[(df['is_promo'] == 'promo_period') 
               & (df['is_promo_asin'] == "promo_asins") 
               & (df['display_price'].isin([4.99, 9.99, 14.99]))]
    
    asin_subset = df[(df['asin'].isin(asin_list['asin']))]
    
    #Here you delete all rows which dont satisfy some condition
    asin_subset = asin_subset.drop(asin_subset[(asin_subset['is_promo'] == "promo_period") 
                                           & (asin_subset['is_promo_asin'] == "promo_asins")
                                          & (~asin_subset['display_price'].isin([4.99,9.99,14.99]))].index)
    return asin_subset

In [202]:
def seasonality_factors(df1,df2):
    """function merges seasonality factors and the output from get_asin_at_price_points"""
    output = pd.merge(left=df1,right=df2,left_on=['month','new_lifecycles'],right_on=['month','new_lifecycles'],how='left')
    return output

In [203]:
def prep_dataset(df):
    
    df['OPS_deseas'] = df['OPS'] / df['factor']
    a = pd.DataFrame(df.groupby(["is_promo","asin"])['week_ending'].size().reset_index())
    key_metrics = pd.DataFrame(df.groupby(["is_promo","asin"])['OPS','OPS_deseas','units'].sum().reset_index())
    m = pd.merge(left=key_metrics,right=a,left_on=['is_promo','asin'],right_on=['is_promo','asin'], how = 'left')
    m['OPS_per_week'] = m['OPS'] / m['week_ending']
    m['OPS_deseas_per_week'] = m['OPS_deseas'] / m['week_ending']
    m['units_per_week'] = m['units'] / m['week_ending']
    
    after = pd.DataFrame(asin_subset[asin_subset['is_promo']=='promo_period'].
                         groupby(["asin",'is_promo'])["display_price"].min().reset_index())
    before = pd.DataFrame(asin_subset[asin_subset['is_promo']=='non_promo_period'].
                          groupby(["asin",'is_promo'])["display_price"].mean().reset_index())
    all_prices = after.append(before, ignore_index=True)
    key_out = pd.merge(left=m,right=all_prices,left_on=['is_promo','asin'],right_on=['is_promo','asin'],how="left")
    key_out = pd.merge(left=key_out,right=after[['asin','display_price']],left_on="asin",right_on="asin",how="left")
    
    return key_out

In [204]:
asin_subset = get_asin_at_price_points(df)

In [205]:
asin_subset = seasonality_factors(asin_subset,sindex_melt)

In [206]:
key_metrics = prep_dataset(asin_subset)

In [94]:
#Overall stats
#key_metrics.groupby("is_promo")['OPS_per_week','units_per_week'].mean()

,OPS_per_week,units_per_week
is_promo,,
non_promo_period,95.376307,6.999844
promo_period,83.669155,14.566081


In [207]:
key_metrics.groupby("is_promo")['OPS_per_week','OPS_deseas_per_week','units_per_week'].mean()

,OPS_per_week,OPS_deseas_per_week,units_per_week
is_promo,,,
non_promo_period,95.376307,95.149557,6.999844
promo_period,83.669155,88.543896,14.566081


In [14]:
#key_metrics.groupby("is_promo")['OPS_per_week','units_per_week'].mean().to_csv("UK_overall.csv")

In [208]:
def get_output(df, metric):
    output = pd.DataFrame(df.groupby(["is_promo",'display_price_y'])
                          ['OPS_per_week','OPS_deseas_per_week','units_per_week'].mean().reset_index())
    
    
    kk = pd.pivot_table(output, index = "display_price_y",
                        columns="is_promo", values = [metric])
    kk.columns = kk.columns.droplevel()
    kk = kk.reset_index()
    kk['change_in_metric'] = (kk['non_promo_period'] - kk['promo_period']) / kk['non_promo_period']
    kk['change_promo_vs_non_promo_in_%'] = (kk['promo_period'] / kk['non_promo_period']-1)*100
    return kk    

## Initialize metrics to break down the dataset by 

In [211]:
dimension = "season_number"
metric1 = "OPS_deseas_per_week"
#If you want to see the old result, just take metric1 = 'OPS_per_week'
metric2 = "units_per_week"

In [210]:
output = get_output(key_metrics,metric1)
output

is_promo,display_price_y,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
0,4.99,68.437069,61.603700,0.099849,-9.984895
1,9.99,124.475017,132.553931,-0.064904,6.490390
2,14.99,283.741195,245.247152,0.135666,-13.566604


In [17]:
#output = get_output(key_metrics,metric1)
#output

is_promo,display_price_y,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
0,4.99,67.831048,58.378608,0.139353,-13.935270
1,9.99,123.303979,124.329203,-0.008315,0.831461
2,14.99,288.155898,233.657400,0.189129,-18.912852


# Evaluate elasticity of demand
<p> E = percentage_change_in_quantity to percentage_change_in_price </p>

In [212]:
def get_change_in_price(df, metric):
    df['per_change_price'] = (df['display_price_x'] 
                                   - df['display_price_y']) / df['display_price_x']
    k = pd.DataFrame(df[df['is_promo'] == "non_promo_period"].
                     groupby("display_price_y")['per_change_price'].mean().reset_index())
    
    f = get_output(df, metric)
    final = pd.merge(left=f,right=k,left_on="display_price_y",right_on="display_price_y",how="left")
    final['elasticity'] = final['change_in_metric'] / final['per_change_price']
    
    mp = pd.merge(left=final, right=get_output(key_metrics,metric1),
                  left_on="display_price_y",right_on = "display_price_y", how = "left")
    mp.columns = ['price_point','avg_non_period_units','avg_promo_units',
                 'change_in_units','change_in_units_in_percentages',
                 'change_in_price','elasticity','non_promo_ops','promo_ops','change_in_ops','change_in_ops_in_percentages']
    return mp

In [213]:
final = get_change_in_price(key_metrics,metric2)
final

,price_point,avg_non_period_units,avg_promo_units,change_in_units,change_in_units_in_percentages,change_in_price,elasticity,non_promo_ops,promo_ops,change_in_ops,change_in_ops_in_percentages
0,4.99,5.840259,14.037450,-1.403566,140.356615,0.646288,-2.171736,68.437069,61.603700,0.099849,-9.984895
1,9.99,8.676344,14.984418,-0.727043,72.704284,0.418980,-1.735267,124.475017,132.553931,-0.064904,6.490390
2,14.99,13.462861,19.598794,-0.455767,45.576743,0.338411,-1.346786,283.741195,245.247152,0.135666,-13.566604


In [20]:
#final.to_csv("output_UK.csv")

# Add dimensions to the analysis

## Here add complexity to get new dimensions

In [21]:
def prep_dataset_dimension(df, dimension):
    """adds complexity to the analysis in the dimension"""
    
    a = pd.DataFrame(df.groupby(["is_promo","asin"])['week_ending'].size().reset_index())
    key_metrics = pd.DataFrame(df.groupby(["is_promo","asin"])['OPS','units'].sum().reset_index())
    m = pd.merge(left=key_metrics,right=a,left_on=['is_promo','asin'],right_on=['is_promo','asin'], how = 'left')
    
    m['OPS_per_week'] = m['OPS'] / m['week_ending']
    m['units_per_week'] = m['units'] / m['week_ending']
    
    lifecycles = pd.DataFrame(asin_subset.groupby(['asin'])[dimension].max().reset_index())
    ops = pd.merge(left=m,right=lifecycles,left_on="asin",right_on="asin",how = "left")
    
    after = pd.DataFrame(asin_subset[asin_subset['is_promo']=='promo_period'].
                         groupby(["asin",'is_promo'])["display_price"].min().reset_index())
    
    before = pd.DataFrame(asin_subset[asin_subset['is_promo']=='non_promo_period'].
                          groupby(["asin",'is_promo'])["display_price"].mean().reset_index())
    
    all_prices = after.append(before, ignore_index=True)
    key_out = pd.merge(left=ops,right=all_prices,left_on=['is_promo','asin'],right_on=['is_promo','asin'],how="left")
    key_out = pd.merge(left=key_out,right=after[['asin','display_price']],left_on="asin",right_on="asin",how="left")
    
    
    return key_out

In [22]:
def get_output_dimension(df, metric, dimension):
    output = pd.DataFrame(df.groupby(["is_promo",'display_price_y',dimension])
                          ['OPS_per_week','units_per_week'].mean().reset_index())
    
    
    kk = pd.pivot_table(output, index = ["display_price_y",dimension],
                        columns="is_promo", values = [metric])
    kk.columns = kk.columns.droplevel()
    kk = kk.reset_index()
    kk['change_in_metric'] = (kk['non_promo_period'] - kk['promo_period']) / kk['non_promo_period']
    kk['change_promo_vs_non_promo_in_%'] = (kk['promo_period'] / kk['non_promo_period']-1)*100
    return kk  

In [23]:
def get_change_in_price_dimension(df, metric, dimension):
    """make output table with additional dimension"""
    
    df['per_change_price'] = (df['display_price_x'] 
                                   - df['display_price_y']) / df['display_price_x']
    k = pd.DataFrame(df[df['is_promo'] == "non_promo_period"].
                     groupby(["display_price_y",dimension])['per_change_price'].mean().reset_index())
    
    f = get_output_dimension(df, metric, dimension)
    
    final = pd.merge(left=f,right=k,
                     left_on=["display_price_y",dimension],
                     right_on=["display_price_y",dimension],how="left")
    final['elasticity'] = final['change_in_metric'] / final['per_change_price']
    return final

In [24]:
print("ASIN subset shape: ", asin_subset.shape)
print("Number of unique ASINs in the dataset: ", asin_subset['asin'].nunique())

ASIN subset shape:  (30791, 19)
Number of unique ASINs in the dataset:  2594


# Run by dimensions

In [25]:
key_metrics_l = prep_dataset_dimension(asin_subset,dimension)
key_metrics_l.head()

,is_promo,asin,OPS,units,week_ending,OPS_per_week,units_per_week,season_number,display_price_x,display_price_y
0,non_promo_period,B00ERIF4RK,3416.45,241,9,379.605556,26.777778,2.0,18.267778,4.99
1,non_promo_period,B00ERIF6S2,90.69,5,5,18.138000,1.000000,3.0,21.582000,4.99
2,non_promo_period,B00ERIFUK6,43.64,5,3,14.546667,1.666667,1.0,10.656667,4.99
3,non_promo_period,B00ERIG3Z2,33.60,2,2,16.800000,1.000000,3.0,19.990000,4.99
4,non_promo_period,B00ERIHD1K,214.53,22,7,30.647143,3.142857,1.0,11.842857,4.99


In [26]:
outputl = get_output_dimension(key_metrics_l,metric2,dimension)
outputl.head()

is_promo,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
0,4.99,0.0,6.347222,20.775000,-2.273085,227.308534
1,4.99,1.0,7.196540,16.772286,-1.330604,133.060407
2,4.99,2.0,4.929498,11.870753,-1.408106,140.810579
3,4.99,3.0,5.349888,14.123084,-1.639884,163.988391
4,4.99,4.0,5.437494,12.736406,-1.342330,134.233025


In [27]:
#COMMENT Distribution of each price point by content age in terms of OPS, number of asins, change in price, units, OPS

final_d = get_change_in_price_dimension(key_metrics_l,metric2,dimension)
#final_d[final_d['season_number'].isin([1.0,2.0,3.0,4.0])]
final_d[final_d['display_price_y'] == 9.99].head()

,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%,per_change_price,elasticity
46,9.99,1.0,5.456611,9.074638,-0.663054,66.305384,0.392328,-1.690048
47,9.99,2.0,7.575694,13.863788,-0.830035,83.003523,0.394730,-2.102795
48,9.99,3.0,6.619553,12.964064,-0.958450,95.845005,0.446675,-2.145742
49,9.99,4.0,9.421849,18.274634,-0.939602,93.960155,0.414377,-2.267504
50,9.99,5.0,17.599795,25.156604,-0.429369,42.936916,0.423813,-1.013110


## Add on to the table

<p> 
For DE, according to the data here, total promo OPS was 2.089m. It is cleaned of all ASINs that were not sold at price points of 4.99, 9.99 or 14.99 representing still over 90% of total OPS in the promo. Now, I dissect the revenue by dimensions, either by content age or by season number. Similar % part is in the UK. </p>

In [28]:
key_metrics_l[key_metrics_l['is_promo'] == 'promo_period']["OPS"].sum()

2089362.5800000001

In [29]:
in_promo = key_metrics_l[key_metrics_l['is_promo'] == 'promo_period']
print(in_promo.shape, in_promo['OPS'].sum())
in_promo.head()

(2594, 11) 2089362.58


,is_promo,asin,OPS,units,week_ending,OPS_per_week,units_per_week,season_number,display_price_x,display_price_y,per_change_price
2289,promo_period,B00ERIF4RK,5862.71,1421,10,586.271000,142.100000,2.0,4.99,4.99,0.0
2290,promo_period,B00ERIF6S2,163.33,41,10,16.333000,4.100000,3.0,4.99,4.99,0.0
2291,promo_period,B00ERIFGY6,4.19,1,1,4.190000,1.000000,3.0,4.99,4.99,0.0
2292,promo_period,B00ERIFUK6,35.59,9,7,5.084286,1.285714,1.0,4.99,4.99,0.0
2293,promo_period,B00ERIG3Z2,87.96,22,9,9.773333,2.444444,3.0,4.99,4.99,0.0


In [30]:
def get_asin_by_dimension(df,dimension):
    t1 = pd.DataFrame(df.groupby(['display_price_y',dimension]).size().reset_index())
    t1.columns = ['display_price_y', dimension, "size"]
    
    t2 = pd.DataFrame(df.groupby(['display_price_y',dimension])['OPS','units'].sum().reset_index())
    t2.columns = ['display_price_y', dimension, "ops",'units']
    
    t = pd.merge(left=t2,right=t1,left_on=['display_price_y',dimension],
                right_on=['display_price_y',dimension], how = 'left')
    
    total = pd.DataFrame(t.groupby("display_price_y")['ops','units'].sum().reset_index())
    total.columns = ['display_price_y','total_ops','total_units']
    t = pd.merge(left=t,right=total,left_on="display_price_y",right_on="display_price_y", how = "left")
    t['share_ops'] = t['ops'] / t['total_ops']
    t['share_units'] = t['units'] / t['total_units']
    return t

In [31]:
asin_dim = get_asin_by_dimension(in_promo,dimension)

In [32]:
asin_dim.head()

,display_price_y,season_number,ops,units,size,total_ops,total_units,share_ops,share_units
0,4.99,0.0,2062.86,492,2,1053397.3,252230,0.001958,0.001951
1,4.99,1.0,430263.55,101011,616,1053397.3,252230,0.408453,0.400472
2,4.99,2.0,155028.24,37508,317,1053397.3,252230,0.147170,0.148706
3,4.99,3.0,122975.60,28659,211,1053397.3,252230,0.116742,0.113622
4,4.99,4.0,72822.27,17848,144,1053397.3,252230,0.069131,0.070761


In [33]:
final_d.head()

,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%,per_change_price,elasticity
0,4.99,0.0,6.347222,20.775000,-2.273085,227.308534,0.632823,-3.591974
1,4.99,1.0,7.196540,16.772286,-1.330604,133.060407,0.602761,-2.207514
2,4.99,2.0,4.929498,11.870753,-1.408106,140.810579,0.652094,-2.159360
3,4.99,3.0,5.349888,14.123084,-1.639884,163.988391,0.661494,-2.479060
4,4.99,4.0,5.437494,12.736406,-1.342330,134.233025,0.674531,-1.990021


In [34]:
def final_join(d1,d2,dimension):
    g= pd.merge(left=d1,right=d2,left_on=['display_price_y',dimension],right_on=['display_price_y',dimension],how='left')
    g['avg_ops_old'] = g['display_price_y'] / g['per_change_price'] * g['non_promo_period']
    g['avg_ops_new'] = g['display_price_y'] * g['promo_period']
    g['ops_diff_in_%'] = (g['avg_ops_new'] - g['avg_ops_old']) / g['avg_ops_old']
    return g

In [35]:
fff = final_join(final_d,asin_dim,dimension)

## Add change in the OPS to the table

In [36]:
def stack_ops_to_table(df, metric, dimension):
    ops_change = get_output_dimension(key_metrics_l,metric,dimension)
    ops_change.rename(index=str, columns={"non_promo_period": "non_promo_OPS", "promo_period": "promo_OPS"})
    return ops_change

In [37]:
ops = stack_ops_to_table(key_metrics_l, metric1, dimension)
ops[ops['display_price_y'] == 9.99].head()

is_promo,display_price_y,season_number,non_promo_period,promo_period,change_in_metric,change_promo_vs_non_promo_in_%
46,9.99,1.0,71.799534,74.961190,-0.044034,4.403450
47,9.99,2.0,101.002799,117.216634,-0.160529,16.052858
48,9.99,3.0,101.432633,106.771584,-0.052635,5.263544
49,9.99,4.0,130.848867,153.087906,-0.169960,16.995974
50,9.99,5.0,267.257212,210.592300,0.212024,-21.202388


In [38]:
def just_merge(df1,df2, dimension):
    return pd.merge(left=df1,right=df2,left_on=['display_price_y',dimension],right_on=['display_price_y',dimension],how='left')

In [39]:
rff = just_merge(fff,ops, dimension)

In [40]:
print(rff.shape)
rff[rff['display_price_y'] == 4.99].head()

(90, 22)


,display_price_y,season_number,non_promo_period_x,promo_period_x,change_in_metric_x,change_promo_vs_non_promo_in_%_x,per_change_price,elasticity,ops,units,...,total_units,share_ops,share_units,avg_ops_old,avg_ops_new,ops_diff_in_%,non_promo_period_y,promo_period_y,change_in_metric_y,change_promo_vs_non_promo_in_%_y
0,4.99,0.0,6.347222,20.775000,-2.273085,227.308534,0.632823,-3.591974,2062.86,492,...,252230,0.001958,0.001951,50.049727,103.667250,1.071285,73.039861,87.104250,-0.192558,19.255772
1,4.99,1.0,7.196540,16.772286,-1.330604,133.060407,0.602761,-2.207514,430263.55,101011,...,252230,0.408453,0.400472,59.577036,83.693709,0.404798,74.478080,70.934968,0.047573,-4.757256
2,4.99,2.0,4.929498,11.870753,-1.408106,140.810579,0.652094,-2.159360,155028.24,37508,...,252230,0.147170,0.148706,37.721854,59.235060,0.570311,57.848130,48.973969,0.153404,-15.340446
3,4.99,3.0,5.349888,14.123084,-1.639884,163.988391,0.661494,-2.479060,122975.60,28659,...,252230,0.116742,0.113622,40.357023,70.474187,0.746268,70.232219,60.150106,0.143554,-14.355395
4,4.99,4.0,5.437494,12.736406,-1.342330,134.233025,0.674531,-1.990021,72822.27,17848,...,252230,0.069131,0.070761,40.225145,63.554665,0.579974,72.970474,51.954598,0.288005,-28.800520


In [41]:
#rff.to_csv("UK_table_2.csv")

# Sales distribution

In [42]:
# print(df.shape)
# df.head()

In [43]:
# df[(df['is_promo'] == 'promo_period') & (df['is_promo_asin'] == 'promo_asins')]['OPS'].sum()

In [44]:
# def sales_dist(df,dimension):
#     sales_dist = pd.DataFrame(df[(df['is_promo'] == 'promo_period') & (df['is_promo_asin'] == 'promo_asins')].groupby(dimension)['OPS'].sum().reset_index())
#     sales_dist['share'] = sales_dist['OPS'] / sales_dist['OPS'].sum()
#     return sales_dist

In [45]:
# sales_dist_df = sales_dist(df,"lifecycle")

In [46]:
# sales_dist_df.head()

# Check ARPU per user for TV Season

In [47]:
# arpu = pd.read_csv("../TV_test/data/ARPU.txt", sep = "\t", 
#                    names = ['total','new','ops','units','cogs','week','marketplace','content'])
# arpu['week'] = pd.to_datetime(arpu['week']) 

In [48]:
# arpu.head()
# arpu['arpu_per_user'] = arpu['ops'] / arpu['total']
# arpu['year_month'] = arpu['week'].map(lambda x: 1000*x.year + x.month)
# arpu['year'] = arpu['week'].map(lambda x: x.year)
# arpu['month'] = arpu['week'].map(lambda x: x.month)

In [49]:
# arpu  = arpu[arpu['content'] == "TV Season"]

In [50]:
# arpu.head()

In [51]:
# arpu.plot(x = "week", y = 'arpu_per_user', figsize = (20,10))
# plt.show()

In [52]:
# arpu[(arpu['week'] >= "2017-01-01") & (arpu['week'] <= '2017-05-01')].plot(x = "week", y = 'arpu_per_user', figsize = (20,10))

In [53]:
# g = pd.DataFrame(arpu[(arpu['week'] >= "2016-01-01")
#               & (arpu['week'] <= '2017-06-01')].groupby(['year_month'])['ops','total'].sum().reset_index())
# g['arpu_per_user'] = g['ops'] / g['total']
# print("Here is the ARPU per user for TV Season in 2017 per month")
# g

## Breakdown the data by lifecycle

<p> Make prediction by 2 categories: new release  vs deep catalogue, new release 0 to 6m, rest is deep catalogue, promo vs nonpromo </p>
<p> Take seasonality into account </p>